<a href="https://colab.research.google.com/github/csgm2328/Emily/blob/master/Max_Mcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import pandas as pd
from pandas import json_normalize
import numpy as np
import plotly as py
import plotly.graph_objs as go
import requests
from keras import metrics
from keras.layers import *
from keras.models import *
from keras.utils.np_utils import to_categorical
from keras.callbacks import *
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
plt.style.use('bmh')
import os
from keras.models import load_model

In [32]:
start_date = datetime(2017,5,1)
start = int(start_date.timestamp()) #꼭 tuple이었던 변수를 datetime으로 넣을 때는 *을 붙여줘야함
ret = requests.get('https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start={}&end=9999999999&period=86400'.format(start))
print(ret)

<Response [200]>


In [33]:
js = ret.json()
print(js[-1])
df = pd.DataFrame(js)

{'date': 1604448000, 'high': 14034.99999998, 'low': 13532.19398223, 'open': 14014.59395478, 'close': 14022.91059739, 'volume': 11859822.33752275, 'quoteVolume': 859.33529154, 'weightedAverage': 13801.15823739}


In [34]:
scaler = MinMaxScaler()
real_price = df['close'].tolist()
#df['close'] series라서 .iloc[]으로 인덱싱
#df[['close']] dateframe 열이름까지 나옴
price = scaler.fit_transform(df[['close']]).reshape(-1).tolist()  #tolist로 만들어줌

print('오늘은',datetime.today(), real_price[-1], len(real_price), '개') # GMT +0 기준

오늘은 2020-11-04 16:24:44.477439 14022.91059739 1284 개


In [35]:
def make_DataSet(w):
  #make Dataset
  x = []  #5일의 데이터를 넣고 다음날을 예측하는 방식 (특징벡터 - 훈련집합)
  y = []  #5+1일의 데이터를 넣고 (결과값 - 훈련집합)

  for i in range(len(price) - w):   #1916-5 = 1911 Range는 마지막 1910까지임
    x.append([price[i+j] for j in range(w)])
    y.append(price[w + i])
  
  x = np.asarray(x)
  y = np.asarray(y)

  train_test_split = int(len(x)*0.7) #1200개에서 1357개로 증가후에 acc: 0.0017 --> 0.0015

  x_train = x[:train_test_split, :]
  y_train = y[:train_test_split]

  x_test = x[train_test_split:, :]
  y_test = y[train_test_split:]

  xtrain = x_train.reshape((x_train.shape[0], x_train.shape[1], 1)) # 2차원 데이터 3차원화, LSTM의 데이터 생성할때 주의해야하는 포인트, [sequence,  batch, input_dim]
  xtest =  np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  #test 집합을 늘리면 미래를 예측할 수 있을까?

  return xtrain, xtest, y_train, y_test

In [36]:
def make_Prediction(window_size, n_batch):
  x = []  
  for i in range(len(price) - window_size): 
    x.append([price[i+j] for j in range(window_size)])
  x = np.asarray(x)
  train_test_split = int(len(x)*0.7)

  x_test = x[train_test_split:, :]
  xtest =  np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

  testing = model.predict(xtest,batch_size=n_batch)
  model.reset_states()
  real_testing = scaler.inverse_transform(testing)
  return testing, real_testing

In [37]:
#F1 score는 지나치게 긍정적으로 평가되는 경향이 있다. 분류문제인 양수인지 음수인지를 판별할 때는 MCC를 사용한다
#MCC는 -1 ~ +1 사이 값을 나타내며 0은 임의 예측과 같은성능
#방향 정확성 평가, MCC는 상승하락에 대해 균형있게 예측했는지 평가
#Matthews correlation coefficient, confusion Matrix 
def MCC(actual, target, t): 
  TP, TN, FP, FN = 0,0,0,0
  ACC_List = [] 
  MCC_List = []

  for i in range(len(target)):
    if actual[i+t] - actual[i+t-1] > 0: #상승
      if target[i] - actual[i+t-1] > 0: #상승예측
        TP += 1
      else:
        FN += 1
    elif actual[i+t] - actual[i+t-1] < 0: # 하락
      if target[i] - actual[i+t-1] < 0:
        TN += 1
      else:
        FP += 1
    else: #그대로
      if target[i] == actual[i+t-1]:
        TP += 1
      elif target[i] - actual[i+t-1] > 0:
        FP += 1
      else:
        FN += 1
    N = TP + TN + FP + FN
    S = (TP + FN) / N
    P = (TP + FP) / N
    ACC = (TP + TN) / N
    if P == 0 or ACC == 0:
      MCC = np.nan
    else:
      MCC = (TP / N - S * P) / np.sqrt(P*S*(1-S)*(1-P))
    ACC_List.append(ACC)
    MCC_List.append(MCC)

  N = TP + TN + FP + FN
  S = (TP + FN) / N
  P = (TP + FP) / N
  ACC = (TP + TN) / N
  MCC = (TP / N - S * P) / np.sqrt(P*S*(1-S)*(1-P))
  #MCC = (TP * TN - FP * FN) / np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
  #두개가 0이되면 측정안됨
  print(TP,FN,TN,FP)
  #print(ACC_List)
  #print(MCC_List)
  print('방향 정확도: %.3f%%\nMCC: %.2f' %(ACC*100,MCC))
  return ACC, MCC, ACC_List, MCC_List

In [38]:
#keras 라이브러리 내 mape 평가는 값이 너무 큼 mae,mse 전부다
def MAPE(actual, target, t): #Mean absolute percentage error
  scores = []
  for i in range(len(target)):
    scores.append(np.abs((actual[i+t] - target[i]) / actual[i+t]))
  print('값 오차율: %.2f%%\n' %(np.average(scores)*100))
  return scores

In [39]:
#모델마다 돌면서 max mcc, acc list 만들기
def find_max(verif_date, temp, sec):
  max_list = []
  sec_max_pt = np.zeros((len(temp),1))
  max_pt = np.zeros((len(temp),1))
  date = 0 #구간표현용

  for i in range(verif_date): #  검증 날짜수
    max = -9999
    for j in range(len(temp)): #  모델 수   
      if  max < temp[j][i]:
        max = temp[j][i]
      if j == len(temp)-1: # 모델 다 돈 후에 max 값과 같은 값을 가진 모델들에게 sec_max_pt + 1
        for k in range(len(temp)):
          if max == temp[k][i]:
            sec_max_pt[k]+=1

    if max == -9999:
      max = np.nan

    max_list.append(max) #그 날의 가장 우수한 metric값들 저장

    if sec != 0: #구간 평가일때
      if (i+1) % sec == 0: #구간마다 평가
        print('\n<<<<<[%d ~ %d]일 구간 우수모델>>>>>' %(verif_date-date, verif_date-(i+1)))
        date = i+1
        max_rank = sorted(sec_max_pt,reverse=True)
        rank = 5 # 몇위까지?
        for y in range(rank):
          for z in range(len(sec_max_pt)):
            if max_rank[y-1] != max_rank[y] and max_rank[y] == sec_max_pt[z]:
              #z번째의 모델 이름
              print('%d위: [%d점] %s' %(y+1, max_rank[y], model_list[z]))

        max_pt += sec_max_pt
        sec_max_pt = np.zeros((len(temp),1))

  max_pt += sec_max_pt
  print('\n<<<<<%d일 누적 우수모델>>>>>' %verif_date)
  max_rank = sorted(max_pt,reverse=True) 
  rank = 10 
  for y in range(rank):
    for z in range(len(max_pt)):
      if max_rank[y-1] != max_rank[y] and max_rank[y] == max_pt[z]:
        print('%d위: [%d점] %s' %(y+1, max_rank[y], model_list[z]))
        
  return max_list,max_pt

In [40]:
#구간별 평가
#mcc가 초기화될때마다 TP가 0일때 평가가 안되는 문제는 감수해야함
def sec_metrics(sec, verif_date):
  acc, mcc = [],[]
  ACC_List, MCC_List = [],[]
  MAPE_List = []

  for i in range(-verif_date,0,sec): #검증하는 기간을 구간별로 나눠서 반복
    if i+sec == 0:
      sec_acc, sec_mcc, sec_ACC_List, sec_MCC_List = MCC(real_price[i:],real_testing[i:],0)
      sec_MAPE_List = MAPE(real_price[i:], real_testing[i:], 0)
    else:
      sec_acc, sec_mcc, sec_ACC_List, sec_MCC_List = MCC(real_price[i:i+sec],real_testing[i:i+sec],0)
      sec_MAPE_List= MAPE(real_price[i:i+sec], real_testing[i:i+sec], 0)

    acc.append(sec_acc)
    mcc.append(sec_mcc)
    MAPE_List.append(sec_MAPE_List)  
    ACC_List.append(sec_ACC_List)
    MCC_List.append(sec_MCC_List)

  #sec_MCCs = sum(sec_MCCs,[]) #sum(iterable, start) np.flatten() 보다 1000배 느림 %timeit으로 검사
  #1차원으로
  acc = np.asarray(acc).flatten()
  mcc = np.asarray(mcc).flatten()
  MAPE_List = np.asarray(MAPE_List).flatten()
  ACC_List = np.asarray(ACC_List).flatten()
  MCC_List = np.asarray(MCC_List).flatten()

  return acc, mcc, MAPE_List, ACC_List, MCC_List
  

In [41]:
cd /content/drive/My Drive/Colab Notebooks/LSTM_models/BTC-USD

/content/drive/My Drive/Colab Notebooks/LSTM_models/BTC-USD


In [42]:
#모델 저장된 폴더에서 모델들 불러오기
model_list = !ls
#model_list = model_list[:10] #upload 용 output 줄이기
len(model_list)

94

In [43]:
#따옴표 없애기
for i in range(len(model_list)):
  s = model_list[i]
  s = s[1:]
  s = s[:-1]
  model_list[i] = s

In [44]:
#epoch 종류 list
ep_list = []
for i in range(len(model_list)):
  ep = model_list[i].split(' ')[7][:-2]
  if ep not in ep_list:
  #if (np.any(ep == ep_list)):
    ep_list.append(ep)
print(ep_list)

['2000', '100', '5000', '7000', '10', '1000', '4000', '400', '5700', '3500', '2500', '1400', '1300', '200', '2', '300', '500', '600', '6', '700', '800', '900']


#Argument별 평가

In [45]:
#Argument별 평가
%%time
ep_list = []
#epoch 종류별로 window_size에 따른 mae값 비교
for i in range(len(ep_list)):
  train_maes, test_maes = [],[]
  xaxes = []
  for j in range(len(model_list)):
    if ep_list[i] == model_list[j].split(' ')[7][:-2]: #같은 종류의 ep찾으면
      #evaluate
      n_batch = int(model_list[j].split(' ')[6][:-3])
      window_size = int(model_list[j].split(' ')[5][:-3])

      xtrain, xtest, y_train, y_test = make_DataSet(window_size)
      model = load_model(model_list[j])
      trainScore = model.evaluate(xtrain, y_train, batch_size = n_batch, verbose=0)
      testScore = model.evaluate(xtest, y_test, batch_size = n_batch, verbose=0)

      print(model_list[j])
      print('mae: (%.5f, %.5f)\n' %(trainScore[1],testScore[1]))
      xaxes.append(window_size)
      train_maes.append(trainScore[1])
      test_maes.append(testScore[1])
    
  #graph
  #min 값은?
  print(xaxes)
  
  print('train MIN : %f' %(np.min(train_maes)))
  print('test MIN : %f' %(np.min(test_maes)))

  train_ax = plt.subplot()
  test_ax = train_ax.twinx()

  train_ax.plot(xaxes, train_maes, 'yo', label='train_mae')
  test_ax.plot(xaxes, test_maes, 'bo', label='test_mae')

  train_ax.set_xlabel(ep_list[i]+'ep: window_size')
  #train_ax.set_ylabel('train_mae')
  #test_ax.set_ylabel('test_mae')

  train_ax.legend(loc='best')
  test_ax.legend(loc='best')
   
  plt.show()

  
  

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 49.4 µs


#학습한 모델들 MCC 평가

In [ ]:
#ACC, MCC 평가
%%time
MCCs, ACCs = [],[]
ACC_Lists, MCC_Lists = [],[]
ACC_data, MCC_data = [],[]
MAPE_Lists = []
verif_date = 270
sec = 0  #원하는 매매주기를 입력하세요!

for i in range(len(model_list)):
  model_name = model_list[i]
  model = load_model(model_name)
  window_size = int(model_name.split(' ')[5][:-3])
  n_batch = int(model_name.split(' ')[6][:-3])
  n_epoch = int(model_name.split(' ')[7][:-2])
  name = model_name.split(' ')[1] + ' ~ ' + model_name.split(' ')[3]+ ' ' + str(window_size) + 'win ' + str(n_epoch) + 'ep'


  xtrain, xtest, y_train, y_test = make_DataSet(window_size)
  testing = model.predict(xtest,batch_size=n_batch)
  model.reset_states()
  real_testing = scaler.inverse_transform(testing)
  #testing, real_testing = make_Prediction(window_size, n_batch)
  
  #출력
  print(name)
  if sec != 0: #구간 평가일때
    acc, mcc, MAPE_List, ACC_List, MCC_List = sec_metrics(sec, verif_date)

  else: #누적 평가일때
    acc, mcc, ACC_List,MCC_List = MCC(real_price[-verif_date:], real_testing[-verif_date:], 0)
    MAPE_List = MAPE(real_price[-verif_date:], real_testing[-verif_date:], 0)

  ACCs.append(acc)
  MCCs.append(mcc)
  MAPE_Lists.append(MAPE_List)
  ACC_Lists.append(ACC_List) 
  MCC_Lists.append(MCC_List)

  #make go.scatter data
  x1 = np.arange(0,verif_date,1)
  if acc > 0.5: #50% 이상만 그리기위해
    go_acc = go.Scatter(x=x1, y = ACC_List, line=dict(color='black'), name = name+'[acc]', yaxis='y')
    ACC_data.append(go_acc)
  if mcc > 0.0: 
    go_mcc = go.Scatter(x=x1, y = MCC_List, mode='lines+markers',line=dict(color='blue'), name = name+'[MCC]', yaxis='y2')
    MCC_data.append(go_mcc)

  #go_mape = go.Scatter(x=x1, y = MAPE_List, line=dict(color='yellow'), name = name+'[MAPE]', yaxis='y3')
  #MCC_data.append(go_mcc)
  
#Max찾기
#평균 Max찾기
rank = 10
print('\n<<<<<%d일 누적평균 정확성 우수모델>>>>>' %verif_date)
max_ACCs = sorted(ACCs,reverse=True)
for i in range(rank):
  for j in range(len(ACCs)):
    if max_ACCs[i-1] != max_ACCs[i] and max_ACCs[i] == ACCs[j]:
      print('%d위는 ACC:%.3f%% 의 %s' % (i+1, max_ACCs[i]*100, model_list[j])) # 정확도

print('\n\n<<<<<%d일 누적평균 MCC 우수모델>>>>>' %verif_date)
max_MCCs = sorted(MCCs,reverse=True)
for i in range(rank):
  for j in range(len(MCCs)):
    if max_MCCs[i-1] != max_MCCs[i] and max_MCCs[i] == MCCs[j]:
      print('%d위는 MCC:%.2f 의 %s' % (i+1, max_MCCs[i], model_list[j])) # 정확도
            
acc_max_list,acc_max_pt = find_max(verif_date, ACC_Lists, sec)
mcc_max_list,mcc_max_pt = find_max(verif_date, MCC_Lists, sec)
#mape_max_list = find_max(verif_date, MAPE_Lists)

go_am = go.Scatter(x=x1, y = acc_max_list, line=dict(color='red',width=6),name = 'Max Accuracies', yaxis='y')
go_mm = go.Scatter(x=x1, y = mcc_max_list, mode='lines+markers',line=dict(color='lime',width=6), name = 'Max MCCs', yaxis='y2')
#go_mp = go.Scatter(x=x1, y = mcc_max_list,line=dict(color='magenta',width=6), name = 'Max MAPEs', yaxis='y3')

In [ ]:
[2020-05-01 ~ 2020-07-28] 24win 1000ep
12 137 113 8
방향 정확도: 46.296%
MCC: 0.03
값 오차율: 2.89%

[2020-05-01 ~ 2020-07-29] 24win 100ep
114 35 33 88
방향 정확도: 54.444%
MCC: 0.04
값 오차율: 2.44%

[2020-05-01 ~ 2020-07-30] 12win 10ep
136 13 28 93
방향 정확도: 60.741%
MCC: 0.20
값 오차율: 2.64%

[2020-05-01 ~ 2020-07-31] 12win 2500ep
1 148 119 2
방향 정확도: 44.444%
MCC: -0.05
값 오차율: 2.90%


<<<<<270일 누적평균 정확성 우수모델>>>>>
1위는 ACC:63.333% 의 BTC-UTD: [2014-07-18 ~ 2020-09-02] 128cell 64win 1bat 7000ep 0.2drop adam 0.0001lr 86400period.h5
2위는 ACC:60.741% 의 BTC-UTD: [2020-05-01 ~ 2020-07-30] 128cell 12win 1bat 10ep 0.3drop Adam 0.001lr 1800period.h5
3위는 ACC:59.630% 의 BTC-UTD: [2014-07-18 ~ 2020-09-02] 128cell 96win 1bat 10ep 0.2drop adam 0.0001lr 86400period.h5
4위는 ACC:59.259% 의 BTC-UTD: [2014-07-18 ~ 2020-09-07] 128cell 70win 1bat 5700ep 0.2drop adam 0.0001lr 86400period.h5
5위는 ACC:56.667% 의 BTC-UTD: [2014-07-18 ~ 2020-07-17] 128cell 60win 1bat 100ep 0.2drop Adam 0.001lr.h5
5위는 ACC:56.667% 의 BTC-UTD: [2014-07-18 ~ 2020-09-04] 128cell 32win 1bat 4000ep 0.2drop adam 0.0001lr 86400period.h5
5위는 ACC:56.667% 의 BTC-UTD: [2014-07-18 ~ 2020-09-09] 128cell 140win 1bat 2000ep 0.2drop adam 0.0001lr 86400period.h5
8위는 ACC:56.296% 의 BTC-UTD: [2017-05-01 ~ 2020-08-17] 128cell 60win 1bat 10ep 0.2drop Adam 0.001lr 86400period.h5
8위는 ACC:56.296% 의 BTC-UTD: [2017-05-01 ~ 2020-08-18] 128cell 48win 1bat 100ep 0.2drop Adam 0.001lr 86400period.h5
10위는 ACC:55.926% 의 BTC-UTD: [2014-07-18 ~ 2020-08-28] 128cell 96win 1bat 5000ep 0.2drop adam 0.0001lr 86400period.h5
10위는 ACC:55.926% 의 BTC-UTD: [2014-07-18 ~ 2020-09-03] 128cell 16win 1bat 10ep 0.2drop adam 0.001lr 86400period.h5
10위는 ACC:55.926% 의 BTC-UTD: [2017-05-01 ~ 2020-08-13] 128cell 10win 1bat 100ep 0.2drop Adam 0.001lr 86400period.h5
10위는 ACC:55.926% 의 BTC-UTD: [2017-05-01 ~ 2020-08-13] 128cell 25win 1bat 100ep 0.2drop Adam 0.001lr 86400period.h5
10위는 ACC:55.926% 의 BTC-UTD: [2017-05-01 ~ 2020-08-20] 128cell 42win 1bat 100ep 0.2drop Adam 0.001lr 86400period.h5
10위는 ACC:55.926% 의 BTC-UTD: [2017-05-01 ~ 2020-08-21] 128cell 96win 1bat 400ep 0.2drop Adam 0.001lr 86400period.h5


<<<<<270일 누적평균 MCC 우수모델>>>>>
1위는 MCC:0.26 의 BTC-UTD: [2014-07-18 ~ 2020-09-02] 128cell 64win 1bat 7000ep 0.2drop adam 0.0001lr 86400period.h5
2위는 MCC:0.20 의 BTC-UTD: [2020-05-01 ~ 2020-07-30] 128cell 12win 1bat 10ep 0.3drop Adam 0.001lr 1800period.h5
3위는 MCC:0.19 의 BTC-UTD: [2014-07-18 ~ 2020-09-07] 128cell 70win 1bat 5700ep 0.2drop adam 0.0001lr 86400period.h5
4위는 MCC:0.17 의 BTC-UTD: [2014-07-18 ~ 2020-09-02] 128cell 96win 1bat 10ep 0.2drop adam 0.0001lr 86400period.h5
5위는 MCC:0.16 의 BTC-UTD: [2017-05-01 ~ 2020-07-17] 128cell 60win 1bat 2000ep 0.2drop Adam 0.001lr.h5
6위는 MCC:0.15 의 BTC-UTD: [2014-07-18 ~ 2020-09-04] 128cell 32win 1bat 4000ep 0.2drop adam 0.0001lr 86400period.h5
7위는 MCC:0.13 의 BTC-UTD: [2017-05-01 ~ 2020-08-12] 128cell 40win 1bat 100ep 0.3drop Adam 0.001lr 86400period.h5
8위는 MCC:0.13 의 BTC-UTD: [2017-05-01 ~ 2020-08-14] 128cell 99win 1bat 100ep 0.2drop Adam 0.001lr 86400period.h5
8위는 MCC:0.13 의 BTC-UTD: [2017-05-01 ~ 2020-08-20] 128cell 96win 1bat 2ep 0.2drop Adam 0.001lr 86400period.h5
10위는 MCC:0.12 의 BTC-UTD: [2017-05-01 ~ 2020-07-24] 128cell 65win 1bat 2000ep 0.2drop Adam 0.0001lr.h5

<<<<<270일 누적 우수모델>>>>>
1위: [248점] BTC-UTD: [2014-07-18 ~ 2020-09-02] 128cell 64win 1bat 7000ep 0.2drop adam 0.0001lr 86400period.h5
2위: [26점] BTC-UTD: [2017-05-01 ~ 2020-08-17] 128cell 72win 1bat 1400ep 0.2drop Adam 0.001lr 86400period.h5
3위: [18점] BTC-UTD: [2014-07-18 ~ 2020-09-02] 128cell 96win 1bat 10ep 0.2drop adam 0.0001lr 86400period.h5
4위: [16점] BTC-UTD: [2014-07-18 ~ 2020-09-07] 128cell 70win 1bat 5700ep 0.2drop adam 0.0001lr 86400period.h5
5위: [9점] BTC-UTD: [2017-05-01 ~ 2020-08-20] 128cell 96win 1bat 600ep 0.2drop Adam 0.001lr 86400period.h5
6위: [7점] BTC-UTD: [2014-07-18 ~ 2020-08-28] 128cell 96win 1bat 5000ep 0.2drop adam 0.0001lr 86400period.h5
6위: [7점] BTC-UTD: [2017-05-01 ~ 2020-08-12] 128cell 40win 1bat 100ep 0.3drop Adam 0.001lr 86400period.h5
8위: [6점] BTC-UTD: [2017-01-01 ~ 2020-11-03] 128cell 64win 1bat 7000ep 0.2drop adam 0.0001lr 86400period.h5
9위: [5점] BTC-UTD: [2017-05-01 ~ 2020-08-20] 128cell 96win 1bat 200ep 0.2drop Adam 0.001lr 86400period.h5
9위: [5점] BTC-UTD: [2017-05-01 ~ 2020-08-21] 128cell 96win 1bat 300ep 0.2drop Adam 0.001lr 86400period.h5
9위: [5점] BTC-UTD: [2017-05-01 ~ 2020-08-28] 128cell 96win 1bat 2000ep 0.2drop adam 0.0001lr 86400period.h5

<<<<<270일 누적 우수모델>>>>>
1위: [216점] BTC-UTD: [2014-07-18 ~ 2020-09-02] 128cell 64win 1bat 7000ep 0.2drop adam 0.0001lr 86400period.h5
2위: [22점] BTC-UTD: [2014-07-18 ~ 2020-09-02] 128cell 96win 1bat 10ep 0.2drop adam 0.0001lr 86400period.h5
3위: [13점] BTC-UTD: [2020-05-01 ~ 2020-07-30] 128cell 12win 1bat 10ep 0.3drop Adam 0.001lr 1800period.h5
4위: [12점] BTC-UTD: [2014-07-18 ~ 2020-09-07] 128cell 70win 1bat 5700ep 0.2drop adam 0.0001lr 86400period.h5
5위: [10점] BTC-UTD: [2017-05-01 ~ 2020-08-17] 128cell 72win 1bat 1400ep 0.2drop Adam 0.001lr 86400period.h5
6위: [5점] BTC-UTD: [2014-07-18 ~ 2020-08-28] 128cell 96win 1bat 5000ep 0.2drop adam 0.0001lr 86400period.h5
7위: [4점] BTC-UTD: [2017-01-01 ~ 2020-11-03] 128cell 64win 1bat 7000ep 0.2drop adam 0.0001lr 86400period.h5
7위: [4점] BTC-UTD: [2017-05-01 ~ 2020-08-20] 128cell 96win 1bat 200ep 0.2drop Adam 0.001lr 86400period.h5
7위: [4점] BTC-UTD: [2017-05-01 ~ 2020-08-20] 128cell 96win 1bat 600ep 0.2drop Adam 0.001lr 86400period.h5
7위: [4점] BTC-UTD: [2017-05-01 ~ 2020-08-21] 128cell 96win 1bat 300ep 0.2drop Adam 0.001lr 86400period.h5
CPU times: user 3min 58s, sys: 11.7 s, total: 4min 10s
Wall time: 6min 38s

In [47]:
cd /content/

/content


In [50]:
#합쳐서 acc, mcc
Title = '정확도, MCC의 MAX 찾기' + ' 최근' + str(verif_date) + '일 '+ '[' + str(datetime.date(datetime.today())) + ']'
fig = go.Figure()
fig.add_traces(go_am)
fig.add_traces(go_mm)
fig.add_traces(ACC_data)
fig.add_traces(MCC_data)
# for i in range(len(ACC_data)): #mcc 0 , ACC 50% 이상만 출력 max에 참여해도 없어지는 값 있음
#   if (np.any(ACCs[i]> 0.5) ):
#     fig.add_traces(ACC_data[i])
#   if (np.any(MCCs[i]> 0.0) ):  
#     fig.add_traces(MCC_data[i])

layout = go.Layout(title=Title,
                   yaxis=dict(
                       title='정확도',zeroline=True, showline=True),
                   yaxis2=dict(
                       title = 'MCC', overlaying='y', side='right', position=1)
)
fig.update_layout(layout)
py.offline.plot(fig)


os.rename('temp-plot.html', '합친 ' + Title + '.html')

In [51]:
#모델별로 acc & mcc 
from plotly.subplots import make_subplots
rows = 4
cols = 5
fig = make_subplots(rows=rows, cols=cols)
k = 0

for i in range(rows):
  for j in range(cols):
    fig.add_trace(ACC_data[k],row=i+1, col=j+1)
    #fig.update_yaxes(range=[0,1],row = i+1, col=i+1)
    fig.add_trace(MCC_data[k],row=i+1, col=j+1 )
    # layout = go.Layout(
    #               yaxis=dict(
    #                   title='정확도', zeroline=True, showline=True),
    #               yaxis2=dict(
    #                   title = 'MCC', overlaying='y', side='right',position=1/cols)
    #               )
    #fig.update_layout(layout)
    k+=1

#fig.add_trace(go_am, row = 3, col=3)
#fig.add_trace(go_mm, row = 3, col=3)
fig.update_layout(go.Layout(title=Title))
py.offline.plot(fig)

os.rename('temp-plot.html', '모델별 ' + Title + '.html')

In [52]:
# #모델별로 acc & mcc 
# from plotly.subplots import make_subplots
# rows = 4
# cols = 5

# specs =[]
# for i in range(rows):
#   specs.append([{"secondary_y": True}, {"secondary_y": True},{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}])

# fig = make_subplots(rows=rows, cols=cols,
#                     specs=specs)
# k = 0

# for i in range(rows):
#   for j in range(cols):
#     fig.add_trace(ACC_data[k],row=i+1, col=j+1, secondary_y=False)
#     #fig.update_xaxes(title_text = 'model_name')
#     fig.update_yaxes(range=[0.2,1],row = i+1, col=i+1)
#     fig.add_trace(MCC_data[k],row=i+1, col=j+1 ,secondary_y=True)
#     fig.update_yaxes(secondary_y=[-0.6,1],row = i+1, col=i+1)
#     layout = go.Layout(
#                   yaxis=dict(
#                       title= '정확도', zeroline=True, showline=True),
#                   yaxis2=dict(
#                       title = 'MCC', overlaying='y', side='right',position=1/cols)
#                   )
#     fig.update_layout(layout)
#     k+=1

# #fig.add_trace(go_am, row = 3, col=3)
# #fig.add_trace(go_mm, row = 3, col=3)
# #fig.update_layout(go.Layout(title=Title))
# #py.offline.plot(fig)
# #fig[0].set_title('sub1')
# fig
# #os.rename('temp-plot.html', '모델별 ' + Title + '.html')

In [53]:
#그냥 subplot으로 찍기
# for i in range(len(ACC_Lists)):
#   acc_ax = plt.subplot()
#   mcc_ax = acc_ax.twinx()

#   acc_ax.plot(ACC_Lists[i], 'y')
#   mcc_ax.plot(MCC_Lists[i], 'b')

#   acc_ax.set_xlabel('model')
#   acc_ax.set_ylabel('acc')
#   mcc_ax.set_ylabel('mcc')
#   acc_ax.set_ylim(0.2,1)
#   mcc_ax.set_ylim(-0.6,1)
#   plt.show()

In [54]:
#MCC_Lists[0]

In [55]:
def load(mo_name,verif_date):
  model = load_model(mo_name)
  window_size = int(mo_name[43:45])
  n_batch = int(mo_name[49])
  train_test_split = int(len(price)*0.7)
  st = train_test_split + window_size

  x = []  #5일의 데이터를 넣고 다음날을 예측하는 방식 (특징벡터 - 훈련집합)
  for i in range(len(price) - window_size):   #1916-5 = 1911 Range는 마지막 1910까지임
    x.append([price[i+j] for j in range(window_size)])
  x = np.asarray(x)
  
  st = train_test_split + window_size
  x_test = x[train_test_split:, :]
  xtest =  np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

  testing = model.predict(xtest,batch_size=n_batch)
  model.reset_states()
  real_testing = scaler.inverse_transform(testing)

  return MCC(real_price[-verif_date:], real_testing[-verif_date:], 0)
  

In [56]:
#MCC list
#al,ml = MCC(real_verif_price, real_verif_after_pred[:verif_gap], split_pt+len(test_pred))
#al,ml = MCC(real_price, real_train_pred, window_size)
def mcc_plot(verif_date):
  for i in range(len(model_list)):
    mo_name = model_list[i]
    name = '[' + mo_name[27:32] + ' ' + (mo_name[43:45]) + 'win ' + mo_name[54:58] +']'
    ACC_List, MCC_List = load(mo_name, verif_date)
    acc_t,mcc_t = [],[]

    t = go.Scatter(x=np.arange(0,verif_date,1), y = ACC_List, mode='lines', name = name+'[acc]')
    t1 = go.Scatter(x=np.arange(0,verif_date,1), y = MCC_List, mode='lines', name = name+'[MCC]')
    acc_t.append(t)
    mcc_t.append(t1)

  acc_max_list = find_max(verif_date, acc_t)
  mcc_max_list = find_max(verif_date, mcc_t)

  t2 = go.Scatter(x=np.arange(0,verif_date,1), y = acc_max_list, mode='lines', name = '[acc MAX]')
  t3 = go.Scatter(x=np.arange(0,verif_date,1), y = mcc_max_list, mode='lines', name = '[MCC MAX]')

  layout = go.Layout(Title='MAX acc,MCC 찾기')
  data = [acc_t, mcc_t, t2, t3]
  fig = go.Figure(data=data, layout=layout)
  py.offline.iplot(fig)
